# 한경 컨센서스에서 증권 애널리스트들의 소견을 바탕으로 미래주가 예측하기

In [15]:
# Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

# tqdm : for문 돌릴 때 진행상황
from tqdm import tqdm_notebook

# warning 메시지 무시하는 코드
import warnings ; warnings.filterwarnings(action='ignore')

In [16]:
# 크롬창 띄우기
driver = webdriver.Chrome("chromedriver.exe")  # 크롬 드라이버 로드
driver.get("http://consensus.hankyung.com/apps.analysis/analysis.list?skinType=business&sdate=2002-02-01&edate=2021-01-20")  #한경컨센서스-기업 페이지 로드
time.sleep(3)

In [4]:
# 제목(종목명)으로 검색 결과 지정
element = driver.find_element_by_xpath('//*[@id="_search_value"]').click()
element = driver.find_element_by_xpath('//*[@id="search_value_select"]/a[1]').click()

In [5]:
# 80개 보이게 설정
driver.find_element_by_xpath("""//*[@id="contents"]/div[1]/div/ul/li[3]/a""").click( )

In [6]:
#주식 종목 리스트
theme_list = ["자동차", "코로나19", "스마트카", "2차전지(소재/부품)", "GTX", "영상콘텐츠", "2차전지(생산)", "반도체(재료/부품)", "증권", "줄기세포"]
stock_list = ["011930", "003070"]

#다원
stock_list1 = ["005380","000270","011930", "003070","012330", "011070" ] #페이지수 : 42, 32 ,1, 2, 25 ,21  

#준규
stock_list2 = ["005490", "011790", "000720", "010140", "035760", "253450"]  #페이지수 : 33, 12, 24, 19, 18 ,4

#유경
stock_list3 = ["051910", "006400", "036490", "357780", "006800", "039490", "028300", "003000"]  #페이지수 : 32, 30 11, 1, 10, 10, 1, 2


In [7]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

In [8]:
save = pd.DataFrame({'dateCreated': [],'title' : [], 'contents' : [], 'reasonablePrice' : [] , 'investmentOpinion' : [] , 'writter' :[] , 'providedSource':[]})

In [9]:
save

,dateCreated,title,contents,reasonablePrice,investmentOpinion,writter,providedSource


# 스크롤

In [ ]:
for i in stock_list:
    # 주식 검색 
    element = driver.find_element_by_id("search_text")
    element.clear()
    element.send_keys(i)
    element.submit()
    time.sleep(0.5)

    count=1  #페이지 카운트 
    firstpage = True  # 첫번째 페이지 
    lastpage = False  # 마지막 페이지 
    temp_list = []     # 스크롤 리스트
    prior_date ="2021-01-23" #날짜 비교  # 코드 돌릴때마다 '오늘' 날짜로 setting 

    while count:
        table=driver.find_element_by_class_name("table_style01")

        tbody = table.find_element_by_tag_name("tbody")

        rows=tbody.find_elements_by_tag_name("tr")

        for _, row in enumerate(rows):
            td_obj = row.find_elements_by_tag_name("td")

            #작성일
            dateCreated=td_obj[0].text
            print(dateCreated)

            try:
                element = WebDriverWait(row, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div")))

            except TimeoutException:
                print("해당 페이지가 존재하지않거나, 2초 안에 열리지 않았습니다.")

            finally:
                #제목, 컨텐츠
                hov = ActionChains(driver).move_to_element(row.find_element_by_tag_name('a'))
                hov.perform()
                hov.perform() #한번 더 수행
                time.sleep(0.5)
                _temp=row.find_element_by_css_selector('div').text.split('\n')
                title = _temp[0]
                contents = _temp[1:]
                print(title)

            #적정가격
            reasonablePrice=td_obj[2].text

            #투자의견
            investmentOpinion=td_obj[3].text

            #작성자
            writter=td_obj[4].text
            #제공출처
            providedSource=td_obj[5].text

            #마지막 페이지 확인
            if dateCreated <= prior_date:  
                 #save
                _temp_dict = {'dateCreated':dateCreated,
                                   'title':title,
                                 'contents':contents,
                                 'reasonablePrice':reasonablePrice,
                                     'investmentOpinion':investmentOpinion,
                                  'writter':writter,
                                  'providedSource':providedSource,
                             }
                temp_list.append(_temp_dict)
                prior_date = dateCreated

            else:
                lastpage = True
                print("클릭했수다")
                break
                
        if lastpage == True:  #마지막 페이지면 데이터 저장
            df=pd.DataFrame(temp_list)
            save = pd.concat([save,df])
            print('done')
            scroll_down(driver)
            driver.find_element_by_xpath("""//*[@id="contents"]/div[3]/a[1]""").click( ) 
            break
        else:
            # 페이지 이동
            count = count+1
            if firstpage == True:
                driver.find_element_by_xpath(""" //*[@id="contents"]/div[3]/a["""+(str)(count) +"""]""").click( ) 
                if count==11:
                    firstpage = False
            else:                 
                driver.find_element_by_xpath(""" //*[@id="contents"]/div[3]/a["""+(str)((count-2)%10+3) +"""]""").click( ) 

            time.sleep(0.5)
                
                
                
        

    #time.sleep 넣어줘야 브라우저 에러 방지
    time.sleep(1)
    

2020-04-08
현대차(005380)코로나 영향을 넘어서
2020-04-07
현대차(005380)신차 모멘텀으로 극복하는 코로나
2020-04-07

2020-04-01



In [11]:
save

,dateCreated,title,contents,reasonablePrice,investmentOpinion,writter,providedSource
0,2020-11-02,신성이엔지(011930)2021 년 재생에너지(RE) 사업부 턴어라운드 예상,[2021 년 재생에너지(RE) 사업부 턴어라운드 예상],0,Not Rated,박찬솔,SK증권
1,2020-06-12,신성이엔지(011930)모듈공장 투자로 캐파 더블 업,"[600MW 규모 모듈공장 투자 결정, 2020년 RE사업부(태양광) 흑자전환 예상...",0,Not Rated,함형도,IBK투자증권
2,2020-01-29,신성이엔지(011930)심기일전,[심기일전],0,nr,구성진,DB금융투자
3,2020-01-28,신성이엔지(011930)회춘,"[태양광 본격적인 성장 구간 , 클린룸의 귀환, 2020년 영업이익 281 억원 전망]",0,Not Rated,함형도,IBK투자증권
4,2019-08-16,신성이엔지(011930)태양광부문 흑자는 지속된다,[태양광부문 흑자는 지속된다],0,Not Rated,함형도,IBK투자증권
...,...,...,...,...,...,...,...
145,2005-09-24,코오롱건설 003070,[이익증가는 외형확대와 낮아진 금리 민감도로 2007년까지 지속: 2006년 대형건...,"12,300",BUY,"허문욱,노세연",삼성증권
146,2005-07-20,코오롱건설(003070)BUY(유지),"[적정주가: 10,300원(종전 9,000원) ▶분양호조에 기인하여 상반기 순차입금...","10,300",BUY,황중권,현대증권
147,2005-06-21,코오롱건설 (003070)BUY(H),"[할인요인에 대한 재평가를 반영해 목표주가 9,400원으로 상향조정: 목표주가를 2...","9,400",BUY,"허문욱,노세연",삼성증권
148,2005-06-17,코오롱건설(003070)BUY(유지),"[적정주가: 9,000원(종전 7,000원) ▶동사는 최근 1개월 동안 시장대비 3...","9,000",BUY,황중권,현대증권


In [124]:
# 제목, 적정가격, 투자의견, 작성자, 제공출처 수집

In [12]:
save.to_excel("hk.xlsx")
    